In [ ]:
# %% [markdown]
# ## 1. Import Libraries
#
# We import the necessary libraries for data handling, visualization, and machine learning.

import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Set style for plots
sns.set(style="whitegrid")


In [1]:
# %% [markdown]
# ## 2. Load and Prepare the Data
#
# Here, we load the JSON file and normalize it into a flat DataFrame.
# In this example, we assume the JSON file contains a list of records.

# Replace 'data.json' with the path to your JSON file.
with open('/Users/shrijayan.rajendran/projects/personal/monk/monk_modernbert/dataset/medical_mimic_new_(1) copy.json', 'r') as f:
    data = json.load(f)

# Normalize the nested JSON structure
df = pd.json_normalize(data)

# Display the first few rows to understand the data structure
df.head()


NameError: name 'json' is not defined